# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [63]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.exceptions import *
from sklearn.ensemble import AdaBoostClassifier
import pickle
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
# from xgboost import XGBClassifier

In [2]:
# downloading nltk libraries

# # The line is commented to avoid reinstallation
# nltk.download("popular")

# # Done downloading collection popular

In [33]:
# load data from database
engine = create_engine('sqlite:///NesrCategories.db')
sql = 'SELECT * FROM CategorizedMessages'
df = pd.read_sql(sql, engine)
X = df.message
Y = df.iloc[:, 4:]

# Get Y labels
Y_labels = list(Y)

In [4]:
# Check
display(df.shape, X.shape, Y.shape, 
        df.head(1), X.head(2), Y.head(2))

(26216, 40)

(26216,)

(26216, 36)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [31]:
# Define some shortcuts
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tokenize(text):
    # Normalize text
    text = text.lower()
    text = re.sub(r"[^a-z0-9]", " ", text)
    # Tokenize text
    tokens = word_tokenize(text)
    # lemmatize and Remove stop words
    tokens = [
        lemmatizer.lemmatize(word) for word in tokens if word not in stop_words
    ]
    # Return
    return tokens


def display_single_results(y_test, y_pred, labels=None):
    if labels is None:
        labels = np.unique(y_pred)
    print("Labels: ", labels)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    print("\nConfusion Matrix:\n", confusion_mat)
    accuracy = (y_pred == y_test).mean()
    print(f"\nAccuracy: {accuracy *100:5.2f}%\n")
    weig = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    micr = precision_recall_fscore_support(y_test, y_pred, average='micro')
    macr = precision_recall_fscore_support(y_test, y_pred, average='macro')
    #bina = precision_recall_fscore_support(y_test, y_pred, average='binary')
    print(f"{'Metric':<12}{'Weighted':>10}{'Micro':>10}{'Macro':>10}"
          )  #{'Binary':>10}")
    print(
        f"{'Precision:':<12}{weig[0] *100:9.2f}%{micr[0] *100:9.2f}%{macr[0] *100:9.2f}%"
    )
    print(
        f"{'Recall:':<12}{weig[1] *100:9.2f}%{micr[1] *100:9.2f}%{macr[1] *100:9.2f}%"
    )
    print(
        f"{'F Score:':<12}{weig[2] *100:9.2f}%{micr[2] *100:9.2f}%{macr[2] *100:9.2f}%"
    )

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### 3-1 Trying to build normal Sequence of ML
> *3-1-1 Trying on **one output** only*

In [6]:
# Extracting one output only
y = Y.iloc[:, 0]

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Defining vectorizers and classifiers
print('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = RandomForestClassifier(n_estimators=100)

# train classifier
print('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

# display results
print('Scoring...')
display_single_results(y_test, y_pred)

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
Scoring...
Labels:  [0 1 2]

Confusion Matrix:
 [[ 637  857    1]
 [ 269 4743    5]
 [   1   35    6]]

Accuracy: 82.18%

Metric        Weighted     Micro     Macro
Precision:      80.77%    82.18%    68.13%
Recall:         82.18%    82.18%    50.48%
F Score:        80.41%    82.18%    54.77%


#### 3-1 Trying to build normal Sequence of ML
> *3-1-2 Trying on **multi-outputs**!*

In [41]:
# First, getting a sample of 500 items from the dataframe
# to make testing easier

def get_sample(data_frame, sample_percent=10):
    '''
    Takes a data_frame, takes amount of sample_percent% of it randomly,
    Then returns X and Y (for this project only)
    '''
    n = int(data_frame.shape[0] * sample_percent / 100)
    df = data_frame.sample(n)
    X = df.message
    Y = df.iloc[:, 4:]
    return X, Y

#Test
_X, _Y = get_sample(df, 5)
display(df.shape, _X.shape, _Y.shape, _X.head(2), _Y.head(2))

(26216, 40)

(1310,)

(1310, 36)

332      cyber cafe global univercell cabaret 35 km fro...
19356    The project is designed to boost agriculture t...
Name: message, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
332,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining vectorizers and classifiers
print('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

# The normal classifier
clf_single = RandomForestClassifier(n_estimators=100)
# The multi_output classifier
clf = MultiOutputClassifier(clf_single)

# train classifier
print('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

print('*Done*')
# # display results
# print('Scoring...')
# display_results(y_test, y_pred)

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
*Done*


In [13]:
y_pred.shape, y_test.shape

((197, 36), (197, 36))

In [46]:
# Scoring outputs

# y_pred[:,0]==y_test[:,0]
# (y_pred[:,0] == y_test[:,0]).mean()
# y_test.iloc[:,0]
# y_test.values[:,0]
accuracy = [[(y_pred[:, i] == y_test.values[:, i]).mean(),
             *precision_recall_fscore_support(
                 y_test.values[:, i], y_pred[:, i], average='weighted')]
            for i in range(y_pred.shape[1])]
accuracy = np.array(accuracy)[:, :-1]
accuracy = (accuracy * 10000).astype(int) / 100
pd.DataFrame(
    data=accuracy,
    index=list(y_test),
    columns=['Accuracy', 'Precision', 'Recall', 'F-score'])

,Accuracy,Precision,Recall,F-score
related,77.15,73.29,77.15,71.55
request,87.81,87.78,87.81,84.66
offer,100.00,100.00,100.00,100.00
aid_related,71.57,71.54,71.57,69.90
medical_help,91.87,84.41,91.87,87.98
medical_products,95.93,92.04,95.93,93.95
search_and_rescue,96.44,93.01,96.44,94.70
security,98.98,97.97,98.98,98.47
military,96.44,93.01,96.44,94.70
child_alone,100.00,100.00,100.00,100.00


In [42]:
# Defining a scoring function
def display_multi_output_results(y_test, y_pred, labels=None):
    # Scoring the outputs
    accuracy = [[(y_pred[:, i] == y_test.values[:, i]).mean(),
                 *precision_recall_fscore_support(
                     y_test.values[:, i],
                     y_pred[:, i],
                     average='weighted',
                     labels=np.unique(y_pred[:, i]))]
                for i in range(y_pred.shape[1])]
    accuracy = np.array(accuracy)[:, :-1]
    accuracy = (accuracy * 10000).astype(int) / 100

    print('Showing scores...')
    print('\nAverage scores for all indicators...')
    scores = pd.DataFrame(
        data=accuracy,
        index=list(y_test),
        columns=['Accuracy %', 'Precision %', 'Recall %', 'F-score %'])
    print(scores.mean(axis=0))
    print('\Detailed scores for each indicator...')
    display(scores)
    return scores

In [58]:
# Putting it all together

# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining vectorizers and classifiers
print('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

# The normal classifier
clf_single = RandomForestClassifier(n_estimators=100)
# The multi_output classifier
clf = MultiOutputClassifier(clf_single)

# train classifier
print('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

# Scoring the outputs
display_multi_output_results(y_test, y_pred)

print('*Done*')

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     93.058611
Precision %    92.960000
Recall %       96.295833
F-score %      93.836944
dtype: float64
\Detailed scores for each indicator...


,Accuracy %,Precision %,Recall %,F-score %
related,77.15,73.17,78.35,71.76
request,90.35,91.36,90.35,88.75
offer,100.00,100.00,100.00,100.00
aid_related,73.60,75.21,73.60,72.78
medical_help,90.35,90.35,100.00,94.93
medical_products,95.43,95.43,100.00,97.66
search_and_rescue,95.93,95.93,100.00,97.92
security,97.96,97.96,100.00,98.97
military,96.95,96.95,100.00,98.45
child_alone,100.00,100.00,100.00,100.00


*Done*


In [55]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(n_estimators=100)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [59]:
# Trying on the pipeline

# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining The full pipline
print('Defining the pipeline...')
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(n_estimators=100)))])

# training the pipeline
print('Training the pipeline...')
pipeline.fit(X_train, y_train)

# predicting on test data
print('Performing pipeline predictions...')
y_pred = pipeline.predict(X_test)

# Scoring the outputs
display_multi_output_results(y_test, y_pred)

print('*Done*')

Splitting...
Defining the pipeline...
Training the pipeline...
Performing pipeline predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     93.255000
Precision %    93.246944
Recall %       96.058611
F-score %      93.633611
dtype: float64
\Detailed scores for each indicator...


,Accuracy %,Precision %,Recall %,F-score %
related,77.15,74.60,77.94,73.16
request,83.75,86.42,83.75,77.26
offer,98.47,98.47,100.00,99.23
aid_related,73.60,74.13,73.60,71.48
medical_help,92.38,92.38,100.00,96.04
medical_products,93.40,93.40,100.00,96.58
search_and_rescue,97.46,97.46,100.00,98.71
security,99.49,99.49,100.00,99.74
military,97.96,97.96,100.00,98.97
child_alone,100.00,100.00,100.00,100.00


*Done*


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# I had a better output using pandas than the classification_report

### 6. Improve your model
Use grid search to find better parameters. 

#### 1- Find the model's parameters

In [63]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001EBC699B8C8>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=False,

#### 2. Determine the possible parameters to adjust

In [90]:
# List possible parameters

parameters = {
    'tfidf__norm': ('l2', 'l1'),
    'tfidf__smooth_idf': (True, False),
    'tfidf__sublinear_tf': (False, True),
    'tfidf__use_idf': (True, False),
    'clf__estimator__criterion': ('gini', 'entropy'),
    'clf__estimator__max_depth': (None, 10),
    'clf__estimator__max_features': ('sqrt', 0.3),
    'clf__estimator__n_estimators': (10, 50, 100, 500),
    'clf__estimator__warm_start': (False, True),
    'clf__estimator__min_samples_split': (2, 4),
}


# Number of combinations between parameters
def find_num_combinations(parameters):
    '''A function to find number of combinations between parameters'''
    para_length = 1
    for key, val in parameters.items():
        para_length *= len(val)
    return para_length


# Test
find_num_combinations(parameters)

2048

In [91]:
# testing on another set of parameters
parameters = {
    'tfidf__norm': ('l2', 'l1'),
    'clf__estimator__criterion': ('gini','entropy'),
    'clf__estimator__max_depth': (None, 10),
    'clf__estimator__max_features': ('sqrt', 0.3),
    'clf__estimator__n_estimators': (25, 100, 500),
    'clf__estimator__warm_start': (False, True),
    'clf__estimator__min_samples_split': (2, 4),
}

find_num_combinations(parameters)

192

#### 3. Testing the pipeline on a 3% sample data
> ***Note***, the test took too much time to finish, So I decided to test each parameter separately then to put some of them in combine with eachother after eliminating most of the parameters.

In [96]:
# Trying on the pipeline using Grid search

# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# To ignore SkLearn warnings
# warnings.filterwarnings(action='ignore', category=__all__)
def warn(*args, **kwargs):
    pass

warnings.warn = warn

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining The full pipline
print('Defining the pipeline...')
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(n_jobs=-1, random_state=179, oob_score=True, verbose=1)))])

# Define the parameters
# parameters = {
#     'tfidf__norm': ('l2', 'l1'),
#     'clf__estimator__criterion': ('gini','entropy'),
#     'clf__estimator__max_depth': (None, 10),
#     'clf__estimator__max_features': ('sqrt', 0.3),
#     'clf__estimator__n_estimators': (25, 100, 500),
#     'clf__estimator__warm_start': (False, True),
#     'clf__estimator__min_samples_split': (2, 4),
# }

Splitting...
Defining the pipeline...


In [123]:
def perform_single_param_test(pipeline, parameters):
    '''
    Defines a function that performs GridSearch training of  a 
    {pipeline} with specific 
    {parameters}
    RETURNS:
    the GridSearch {model}
    '''
    # Define the cv model selection
    cv = GridSearchCV(pipeline, param_grid=parameters)

    # training the pipeline
    print('Training the GridSearch pipeline...')
    print(f'Testing combinations of {find_num_combinations(parameters)} parameters')
    cv.fit(X_train, y_train)
    print('* Training is Done *')
    print(cv.best_params_)
    return cv

**Testing the TfidfTransformer `norm` parameter**

In [103]:
parameters = {
    'tfidf__norm': ('l2', 'l1'),
}

model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=

...

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

* Training is Done *
{'tfidf__norm': 'l1'}


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=

In [104]:
model.best_params_

{'tfidf__norm': 'l1'}

*The best pvalue varies between `l1` and `l2`!*
***
**Testing the RandomForestClassifier `criterion` parameter**

In [105]:
warnings.filterwarnings(action='ignore')
parameters = {
    'clf__estimator__criterion': ('gini','entropy'),
}

model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

* Training is Done *
{'clf__estimator__criterion': 'gini'}


[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parall

In [106]:
model.best_params_

{'clf__estimator__criterion': 'gini'}

*The best pvalue varies between `gini` and `entropy`!*
***
**Testing the RandomForestClassifier `n_estimators` parameter**

In [108]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              criterion='entropy',
                              n_jobs=-1,
                              random_state=179,
                              oob_score=True,
                              verbose=1)))])
parameters = {
    'clf__estimator__n_estimators': (25, 100, 500),
}

model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]

[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=

...

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished
[

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBacken

* Training is Done *
{'clf__estimator__n_estimators': 100}


In [109]:
model.best_params_

{'clf__estimator__n_estimators': 100}

In [110]:
# Removing n_jobs and oob_score parameters from the pipeline

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              criterion='entropy',
                              random_state=179,
                              verbose=1)))])
parameters = {
    'clf__estimator__n_estimators': (25, 100, 500),
}

model = perform_single_param_test(pipeline, parameters)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training the GridSearch pipeline...


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

...

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

* Training is Done *
{'clf__estimator__n_estimators': 100}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [111]:
# removing criterion parameter from the pipeline

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              criterion='entropy',
                              random_state=179)))])
parameters = {
    'clf__estimator__n_estimators': (25, 100, 500),
}

model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'clf__estimator__n_estimators': 100}


*The best value is always 100*
***
**Testing the RandomForestClassifier `warm_start` parameter**

In [113]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              criterion='entropy',
                              random_state=179,
                              n_estimators=100)))])
parameters = {
    'clf__estimator__warm_start': (False, True),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'clf__estimator__warm_start': False}


*The best value is always False*
***
**Testing the RandomForestClassifier `max_depth` parameter**

In [125]:

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              criterion='entropy',
                              random_state=179,
                              n_estimators=100)))])
parameters = {
    'clf__estimator__max_depth': (None, 5, 10, 50),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
Testing combinations of 4 parameters
* Training is Done *
{'clf__estimator__max_depth': 5}


*The best value is always 5, but may be less!*
***
**Testing the RandomForestClassifier `max_features` parameter**

In [115]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              criterion='entropy',
                              random_state=179,
                              n_estimators=100)))])
parameters = {
    'clf__estimator__max_features': ('sqrt', 0.3),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'clf__estimator__max_features': 'sqrt'}


*The best value varies between `0.3` and `sqrt`*
***
**Testing the RandomForestClassifier `min_samples_split` parameter**

In [117]:
parameters = {
    'clf__estimator__min_samples_split': (2, 4, 6),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'clf__estimator__min_samples_split': 4}


*The best value is always 4*
***
**Testing the TfidfTransformer `use_idf` parameter**

In [118]:
parameters = {
    'tfidf__use_idf': (True, False),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'tfidf__use_idf': True}


*The best value is always True*
***
**Testing the TfidfTransformer `smooth_idf` parameter**

In [119]:
parameters = {
    'tfidf__smooth_idf': (True, False),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'tfidf__smooth_idf': True}


*The best value is always True*
***
**Testing the TfidfTransformer `sublinear_tf` parameter**

In [121]:
parameters = {
    'tfidf__sublinear_tf': (True, False),
}
model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
* Training is Done *
{'tfidf__sublinear_tf': True}


*The best value varies*
***
***

#### 4. Testing the gridsearch on multiple parameters

In [126]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(
                              random_state=179, n_estimators=100)))])
parameters = {
    'tfidf__sublinear_tf': (True, False),
    'clf__estimator__min_samples_split': (3, 4),
    'clf__estimator__max_features': ('sqrt', 0.2, 0.3),
    'clf__estimator__max_depth': (3, 5),
    'clf__estimator__criterion': ('gini','entropy'),
}
final_model = perform_single_param_test(pipeline, parameters)

Training the GridSearch pipeline...
Testing combinations of 48 parameters
* Training is Done *
{'clf__estimator__criterion': 'entropy', 'clf__estimator__max_depth': 3, 'clf__estimator__max_features': 0.3, 'clf__estimator__min_samples_split': 3, 'tfidf__sublinear_tf': False}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [127]:
# predicting on test data
print('Performing GridSearch pipeline predictions...')
y_pred = final_model.predict(X_test)

# Scoring the outputs
display_multi_output_results(y_test, y_pred)

print('*Done*')

Performing GridSearch pipeline predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     92.859444
Precision %    92.535000
Recall %       94.963611
F-score %      92.700556
dtype: float64
\Detailed scores for each indicator...


,Accuracy %,Precision %,Recall %,F-score %
related,78.17,82.96,78.17,69.08
request,86.29,84.27,86.29,83.63
offer,99.49,99.49,100.00,99.74
aid_related,64.46,67.91,64.46,59.81
medical_help,88.32,78.86,88.32,83.32
medical_products,94.92,94.92,100.00,97.39
search_and_rescue,97.46,97.46,100.00,98.71
security,97.46,97.46,100.00,98.71
military,98.47,98.47,100.00,99.23
child_alone,100.00,100.00,100.00,100.00


*Done*


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# To improve the model I will try both
# AdaBoostClassifier, and DecisionTreeClassifier

In [60]:
def test_sample_with_pipeline(pipeline, percent=3):
    # taking 3% of the original dataframe for faster tests
    _X, _Y = get_sample(df, 3)

    # Split data to train and test
    print('Splitting...')
    X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

    # Defining The full pipline
    print('Defining the pipeline...')
    pipeline = pipeline

    # training the pipeline
    print('Training the pipeline...')
    pipeline.fit(X_train, y_train)

    # predicting on test data
    print('Performing pipeline predictions...')
    y_pred = pipeline.predict(X_test)

    # Scoring the outputs
    display_multi_output_results(y_test, y_pred)

    print('*Done*')
    return pipeline

In [45]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          AdaBoostClassifier(n_estimators=100)))])

pipe = test_sample_with_pipeline(pipeline)

Splitting...
Defining the pipeline...
Training the pipeline...
Performing pipeline predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     92.888333
Precision %    92.345278
Recall %       93.213611
F-score %      92.502222
dtype: float64
\Detailed scores for each indicator...


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,Accuracy %,Precision %,Recall %,F-score %
related,74.11,70.09,74.11,65.97
request,83.24,83.48,83.24,83.36
offer,98.47,98.47,100.00,99.23
aid_related,61.92,61.10,61.92,61.24
medical_help,88.32,86.43,88.32,87.32
medical_products,95.43,95.64,95.43,93.61
search_and_rescue,97.46,97.46,100.00,98.71
security,98.98,98.98,100.00,99.48
military,96.44,96.44,100.00,98.19
child_alone,100.00,100.00,100.00,100.00


*Done*


In [49]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          DecisionTreeClassifier()))])

pipe = test_sample_with_pipeline(pipeline)

Splitting...
Defining the pipeline...
Training the pipeline...
Performing pipeline predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     91.492500
Precision %    91.347778
Recall %       91.549167
F-score %      91.390278
dtype: float64
\Detailed scores for each indicator...


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,Accuracy %,Precision %,Recall %,F-score %
related,72.58,69.84,72.58,70.91
request,77.66,77.66,77.66,77.66
offer,100.00,100.00,100.00,100.00
aid_related,60.91,60.72,60.91,60.78
medical_help,88.83,87.31,88.83,88.03
medical_products,89.84,88.35,89.84,89.05
search_and_rescue,96.44,94.96,96.44,95.69
security,97.46,96.96,97.46,97.21
military,92.38,93.58,92.38,92.95
child_alone,100.00,100.00,100.00,100.00


*Done*


In [67]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(n_estimators=100)))])

pipe = test_sample_with_pipeline(pipeline)

Splitting...
Defining the pipeline...
Training the pipeline...
Performing pipeline predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     93.931944
Precision %    93.849722
Recall %       96.202222
F-score %      94.251667
dtype: float64
\Detailed scores for each indicator...


,Accuracy %,Precision %,Recall %,F-score %
related,75.12,72.00,75.51,70.93
request,85.78,84.41,85.78,83.35
offer,100.00,100.00,100.00,100.00
aid_related,73.09,73.48,73.09,71.10
medical_help,94.92,94.92,100.00,97.39
medical_products,97.46,97.46,100.00,98.71
search_and_rescue,97.46,97.46,100.00,98.71
security,98.98,98.98,100.00,99.48
military,98.47,98.47,100.00,99.23
child_alone,100.00,100.00,100.00,100.00


*Done*


* The RandomForestClassifier gets the best results, so no need to change it
* Regarding the required modifications of `other features besides the TF-IDF`,<br> I found no suitable stemming features, could the reviewer kindly guide me for that?

### 9. Export your model as a pickle file

In [38]:
def model_save_as(model, filename='NesrFittedModel'):
    '''
    Saves the {model} as {filename}
    in pickle format
    Returns:
    None
    '''
    # import os
    # Checking if a File Exists
    if os.path.isfile(f'./{filename}.sav'):
        n = 0
        while os.path.isfile(f'{filename}{n:02d}.sav'):
#             print(f'{filename}{n:02d}.sav',
#                   os.path.isfile(f'{filename}{n:02d}.sav'))
            n += 1
        else:
            filename = f'{filename}{n:02d}.sav'
#             print(filename)
    # save the model to disk
    pickle.dump(model, open(filename, 'wb'))
    print(f'The mode has been saved as: {filename}')
    pass


def model_load(filename):
    '''
    Loads a saved model 
    found in the path {filename}
    in pickle format
    Returns:
    model
    '''
    # load the model from disk
    loaded = pickle.load(open(filename, 'rb'))
    print(f'The mode has been loaded successfully')
    return loaded

In [132]:
# Test to save for the first time
model_save_as(final_model)

The mode has been saved as: NesrFittedModel.sav


In [39]:
# Loading the saved model
final_model = model_load('NesrFittedModel.sav')

The mode has been loaded successfully


In [37]:
# Save Again will add numbers to the filename
model_save_as(final_model)

The mode has been saved as: NesrFittedModel02.sav


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
def build_model():
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer(sublinear_tf=False)),
                    ('clf',
                    MultiOutputClassifier(
                        RandomForestClassifier(n_jobs=1, 
                        n_estimators=100, 
                        random_state=179,
                        criterion='entropy',
                        max_depth=3,
                        max_features=0.3,
                        min_samples_split=3)))])